# Dataset

In [ ]:
from datasets import load_dataset, DatasetDict

# Cargamos el dataset CoNaLa
# Conala es un dataset de pares (descripción en inglés, código en Python)
# lo usaremos para hacer SFT 
train = load_dataset("json", data_files="conala-paired-train.json")['train']
test = load_dataset("json", data_files="conala-paired-test.json")['train']
dataset = DatasetDict({
    "train": train,
    "test": test,
})

In [ ]:
# prompt a usar
PROMPT = """### Instruction:
{instruction}

### Response:
{response}"""

In [ ]:
# Filtramos los ejemplos que no tienen reescritura de la intención
dataset = dataset.filter(lambda x: x["rewritten_intent"] != None)
dataset

In [ ]:
# SFT requiere que el dataset tenga las columnas 'prompt' y 'completion'
# El prompt será la intención reescrita y el completion el snippet de código a predecir
# SFT entrena el modelo para predecir la completion dado el prompt
def preprocess_function(example):
    return {
        "prompt": PROMPT.format(instruction=example['rewritten_intent'], response=""),
        "completion": example['snippet']
    }

dataset = dataset.map(preprocess_function, remove_columns=dataset["train"].column_names)

In [ ]:
print(dataset["test"]["prompt"][0])
print("------------------")
print(dataset["test"]["completion"][0])

# Entrenamiento

In [ ]:
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig
import torch

# Configuramos el entrenamiento
training_args = SFTConfig(
    output_dir="qwen-sft-lora",
    model_init_kwargs={"dtype": torch.bfloat16}, # usar bfloat16 para que el modelo ocupe menos memoria
    report_to="none",
    num_train_epochs=1, # lo dejamos a 1 para que sea rápido
    max_length=256, # es un dataset instrucción + una línea de código, no hace falta que sea muy largo
    per_device_train_batch_size=4, # batch size pequeño para que quepa en GPU
    gradient_accumulation_steps=8, # acumulamos gradientes para simular un batch size mayor
    eval_strategy="epoch", # estrategia de evaluación por pasos, pasos=veces que se modifican los pesos
    save_strategy="epoch", # estrategia de guardado por pasos
    save_total_limit=1, # guardamos solo el último checkpoint para no llenar el disco
    load_best_model_at_end=True, # cargamos el mejor modelo al final del entrenamiento
    learning_rate=3e-4, # tasa de aprendizaje un poco más alta para LoRA
)

# instanciamos el trainer y entrenamos
trainer = SFTTrainer(
    model="Qwen/Qwen2.5-1.5B",
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    args=training_args,
    peft_config=LoraConfig() # usamos LoRA para hacer fine-tuning eficientemente con los parametros por defecto
)
trainer.train()

# Evaluación

In [ ]:
from transformers import pipeline

# probamos el modelo fine-tuneado
pipe = pipeline("text-generation", model="qwen-sft-lora/checkpoint-72", temperature=0.1, do_sample=True)
instruction = "compute the mean of a numpy array"
prompt = PROMPT.format(instruction=instruction, response="")
response = pipe(prompt)

print(response[0]["generated_text"])